# Preprocess IRENA hydro generation data

This notebook preprocesses IRENA data from their "Renewable energy statistics" publication. It reduces the dataset to generation of hydro power in the relevant countries. This notebook creates the `./data/irena/hydro-generation-europe.csv` dataset. The original IRENA dataset is not part of this repository and must be retrieved manually to run this notebook.

In [1]:
import numpy as np
import pandas as pd
import pycountry

In [2]:
def country_id(country_name):
    # fix country names
    if country_name == "UK":
        country_name = "United Kingdom"
    elif country_name == "Bosnia Herzg":
        country_name = "Bosnia and Herzegovina"
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

In [3]:
COUNTRY_IDS = [
    'AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA',
    'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'NLD', 'POL',
    'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'GBR', 'ALB', 'BIH', 'MKD',
    'MNE', 'NOR', 'SRB', 'CHE'
]

In [4]:
df = (
    pd
    .read_excel("../data/IRENA_RE_Statistics_July2020extract.xlsx", index_col=[0, 1, 2])
    .rename_axis(index=["country_code", "indicator", "technology"])
    .rename_axis(columns="year")
    .stack()
    .unstack("indicator")
    .rename(index=country_id, level="country_code")
    .reset_index()
    .dropna("index", subset=["country_code"])
    .set_index(["country_code", "technology", "year"])
    .loc[:, "Electricity generation (GWh)"]
    .to_xarray()
    .sel(technology="Renewable hydropower", country_code=COUNTRY_IDS)
    .to_series()
    .rename("generation_gwh")
    .dropna()
    .sort_index()
    .astype(np.int)
)
df.head()

country_code  year
ALB           2000    4594
              2001    3555
              2002    3512
              2003    4885
              2004    5466
Name: generation_gwh, dtype: int64

In [5]:
df.to_csv("../data/irena/hydro-generation-europe.csv", index=True, header=True)